This scripts retrieve sensors data from the http://archive.luftdaten.info/ 
It uses as input a list of .csv sensors files url and save the resulting data as .csv

**this sript does not work on windows pc because of asyncio**, in this case, you can upload this notebook on google colab.
The asyncio library allows to run in this case 20 threads in parallel, making it much faster to scrap the data.

In [1]:
import datetime
import requests
import lxml.etree
from lxml import html
import pandas as pd
import asyncio
from concurrent.futures import ThreadPoolExecutor
import re
import io
import sqlalchemy
from aiohttp import ClientSession,TCPConnector
import nest_asyncio

import config

# asyncio and jupyter cause trouble, this is a fix :
# https://markhneedham.com/blog/2019/05/10/jupyter-runtimeerror-this-event-loop-is-already-running/
nest_asyncio.apply()

In [5]:
## beautiful asyncio needs beautiful python 3.7
from platform import python_version

print(python_version())

3.7.1


In [2]:
engine = sqlalchemy.create_engine(f"sqlite:///{config.DB_PATH}")

### Load the list of lufdaten sensors (see *luftdaten_geo.ipynb* to get this list)

In [ ]:
df_sensors = pd.read_sql('sensors_BB', engine)
df_sensors['endpoint_url']= df_sensors.sensor_type_name+"_sensor_"+df_sensors.sensor_id.astype(str)+".csv"
df_sensors['endpoint_url']= df_sensors['endpoint_url'].str.lower()
df_sensors.sensor_type_name.unique()

### we set the date range to scrap data (luftdaten data start from 2017)

In [ ]:
def set_dt_range(start='1/1/2017'):
    dt_range = pd.date_range(start, end=datetime.datetime.now().strftime("%Y-%m-%d"))
    dt_range = dt_range.strftime("%Y-%m-%d")
    dt_range = dt_range.tolist()
    return dt_range

In [ ]:
def clean_data(content):
   
    df_data = pd.read_csv(io.StringIO(content),sep=';')
  
    sensor_type = df_data.sensor_type.head(1).values
    
    # we tidy the data because we are cool people :
    if sensor_type  in ['DHT22', 'BME280', 'HTU21D', 'BMP180', 'BMP280']:
        df_data = pd.melt(df_data, id_vars=['sensor_id','timestamp'], value_vars=['temperature','humidity'])
    if sensor_type=="SDS011":
        df_data = pd.melt(df_data, id_vars=['sensor_id','timestamp'], value_vars=['P1','P2'])
        
    return df_data

def aggregate_data(df_data):
    try:
        # aggregate data per hour
        df_data['timestamp'] = pd.to_datetime(df_data['timestamp'])
        df_data['datehour']= df_data['timestamp'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,0))
        df_data = df_data.groupby(['variable','datehour']).mean()

        return df_data
    except Exception as e:
        print(url)
        print(e)
        return None    
  
regex = re.compile("^.*?\.(zip|csv)$")


def get_list_of_files(dt):
    url = f'http://archive.luftdaten.info/{dt}'
    try:
        r = requests.get(url)
        tree = html.fromstring(r.content)
        files = tree.xpath('//a/@href')
        return list(filter(regex.search, files))
    except Exception as e :
        print(e)    

In [ ]:
async def fetch(url, session):
    async with session.get(url) as response:
        content = await response.text()
        df_data = clean_data(content)
        
        return aggregate_data(df_data)

async def run(lista):
   
    tasks = []

    # Fetch all responses within one Client session,
    # keep connection alive for all requests.
    # we set the connector to 10 because we are well educated people who play by the rules :
    connector = TCPConnector(limit=10)
    async with ClientSession(connector=connector) as session:
        for url in lista:
            task = asyncio.ensure_future(fetch(url, session))
            tasks.append(task)

        responses = await asyncio.gather(*tasks,return_exceptions=True )
        # you now have all response bodies in this variable
        return (responses)
    
def scrap_all_data_one_day(dt):
    
    lista = get_list_of_files(dt)

    sensors_available= [l.split('_', 1)[-1] for l in lista]
    sensors_to_retrieve = list(set(sensors_available) & set(all_sensors_BB))

    print(f'{dt}:{len(sensors_to_retrieve)} files to retrieve')

    all_url_of_date = [ROOT_URL.format(dt,dt,l) for l in sensors_to_retrieve]
    
    loop = asyncio.get_event_loop()
    future = asyncio.ensure_future(run(all_url_of_date))
    data = loop.run_until_complete(future)

    df = pd.DataFrame()
    for d in data:
        df = pd.concat([df,d],axis=0)

    return df.reset_index()

def scrap_all_data():
    df_master = pd.DataFrame()

    for i,dt in enumerate(date_range):
      try:
        df = scrap_all_data_one_day(dt)
        df_master=pd.concat([df_master,df],axis=0)

        if i%100==0:
            df_master.to_csv(f'master_{str(i)}.csv')

      except Exception as e:
        print(f'error for {dt}:')
        print(e)

    df_master.to_csv('master_all_.csv')

In [ ]:
ROOT_URL = "http://archive.luftdaten.info/{}/{}_{}"
date_range = set_dt_range()
all_sensors_BB = df_sensors.endpoint_url.tolist()

In [ ]:
scrap_all_data()

In [3]:
df = pd.read_csv('../data/master_all.csv')
print(df.shape)

In [3]:
res = engine.execute("SELECT name FROM sqlite_master WHERE type='table';")

for r in res:
    print(r[0])

stations
data_CO
data_NO2
data_O3
data_PM10
data_SO2
data_CO_1TMW
data_NO2_1TMW
sensors_BB
sensors_BB_data


In [18]:
chunksize = 10 ** 6
#for chunk in pd.read_csv('../data/master_all.csv', chunksize=chunksize):
    #chunk.to_sql(name='sensors_BB_data', con=engine,if_exists='append')

In [5]:
import sqlite3
conn=sqlite3.connect('../data/dwd.db')
conn.execute("VACUUM")
conn.close()

In [ ]:
df_db = pd.read_sql('sensors_BB_data', engine)

In [ ]:
print(df_db.shape)